In [4]:
from typing import Callable

ParserResult = tuple[int, str | None]
Parser = Callable[[str, int], ParserResult]


def word_parser(word: str) -> Parser:
    def parse(s: str, i: int) -> ParserResult:
        if s[i : i + len(word)] == word:
            return i + len(word), word
        return -1, None

    return parse


def sequence(*parsers: Parser) -> Parser:
    def parse(s: str, i: int) -> ParserResult:
        current_index = i
        results = []
        for parser in parsers:
            next_index, result = parser(s, current_index)
            if next_index == -1:
                return (-1, None)
            results.append(result)
            current_index = next_index
        return current_index, results

    return parse


def optional(parser: Parser) -> Parser:
    def parse(s: str, i: int) -> ParserResult:
        index, result = parser(s, i)
        if index == -1:
            return i, None
        return index, result

    return parse


def many(parser: Parser, times: int | None) -> Parser:
    if times is None:
        times = float("inf")  # type: ignore

    def parse(s: str, i: int) -> ParserResult:
        nonlocal times
        current_index = i
        results = []
        while times > 0:
            times -= 1
            next_index, result = parser(s, current_index)
            if next_index == -1:
                return (-1, None)
            results.append(result)
            current_index = next_index
        return current_index, results

    return parse


In [6]:
many(word_parser('hello '), 3)('hello '*13, 0)

(18, ['hello ', 'hello ', 'hello '])